# **COVID-19 Exploratory Data Analysis**
![COVID](https://www.usnews.com/dims4/USNEWS/6b8fce5/2147483647/crop/2000x1333%2B0%2B0/resize/1200x1200%3E/quality/85/?url=https%3A%2F%2Fmedia.beam.usnews.com%2Fd4%2F30%2Fc480b10543ce8443f14e21287493%2F200313-pandemic-stock.jpg)
[Image Credits](https://health.usnews.com/conditions/articles/whats-the-difference-between-an-epidemic-and-pandemic)

COVID-19 (SARS-CoV-2), popularily known as the Coronavirus, was first identified in Wuhan, China in December 2019. Since inception, the infamous virus has caused major disruptions in almost every aspect of life and continues to affect millions all around the globe. 


## Resources on COVID-19
* [World Health Organization (WHO)](https://www.who.int/health-topics/coronavirus#tab=tab_1)
* [US Centers for Disease Control and Prevention (CDC)](https://www.cdc.gov/coronavirus/2019-nCoV/index.html)
* [John Hopkins University](https://coronavirus.jhu.edu/)
* [COVID-19 Tracker for India](https://www.covid19india.org/)

## Acknowledgments
A big shout-out to the amazing community out there for sharing their work. It was truly an invaluable learning experience.
* https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons
* https://www.kaggle.com/tarunkr/covid-19-case-study-analysis-viz-comparisons
* https://www.kaggle.com/abhinand05/covid-19-digging-a-bit-deeper


## <font style="color:#D99B0B "> *Please feel free to comment any feedback you may have on how the notebook can be improved. <font style="color:#DC1818"><b>Stay Home! Stay Safe!</b>*




In [ ]:
#Installations
!pip install calmap


In [ ]:
#Import necessary packages

import numpy as np # linear algebra
import pandas as pd # data processing
import requests
import json

#Visualization Libraries

import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import calmap

from IPython.core.display import HTML

%matplotlib inline
pio.templates.default = "plotly_dark"

### US County Data from New York Times GitHub URL
<hr>

In [ ]:
#New York Times GitHub Data
nyturl = 'https://api.github.com/repos/nytimes/covid-19-data/contents'
r = requests.get(nyturl)
if(r.ok):
    repo = json.loads(r.text or r.content)
    
repo[2]

d = {}

for i in range(len(repo)):
    htmlurl = repo[i]['download_url']
    name = repo[i]['name']
    if name.find(".csv") != -1:
        tempdf = pd.read_csv(htmlurl)
        d["df_" + name] = tempdf
        
d.keys()



### Read Data
<hr>

In [ ]:
#Read data
df = pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv")
US = pd.read_csv("../input/corona-virus-report/usa_county_wise.csv")
nyt_us_counties = d['df_us-counties.csv']
nyt_us_states = d['df_us-states.csv']

In [ ]:
# #Weather history test code
# !pip install wwo-hist
# from wwo_hist import retrieve_hist_data

# frequency=3
# start_date = '11-DEC-2018'
# end_date = '11-MAR-2019'
# api_key = '5601834ed1f74cddb2402604201903'
# location_list = ['singapore','california']

# hist_weather_data = retrieve_hist_data(api_key,
#                                 location_list,
#                                 start_date,
#                                 end_date,
#                                 frequency,
#                                 location_label = False,
#                                 export_csv = False,
#                                 store_df = True)


# singaporetemp = hist_weather_data[0]
# singaporetemp.head()
# singaporetemp[['date_time', 'maxtempC']].groupby('date_time').sum()

# import seaborn as sns
# sns.scatterplot(x= 'date_time', y= 'maxtempC', data = singaporetemp)

### Clean & Format Variables
<hr>

In [ ]:
# #Library - Pandas Profiling 
# import pandas_profiling as pp

# profile = pp.ProfileReport(df,title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile

#Clean Data & Change data types 

df['Date'] = pd.to_datetime(df['Date'])
US['Date'] = pd.to_datetime(US['Date'])

#Create Variables

df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']
US['Active'] = US['Confirmed'] - US['Deaths']

US.info()

### Number of Countries Affected Over Time (Animated Line Plot)
<hr>

In [ ]:
#Number for Countries affected over time

dfpivot = df.pivot_table(index = 'Country/Region', columns = 'Date', values = ['Confirmed'])

countrycount =  dfpivot.groupby("Country/Region").sum().apply(lambda x: x[x > 0].count(), axis =0)
countrycount = pd.DataFrame(countrycount)

countrycount.reset_index(inplace = True)
countrycount.columns = ["Metric", "Date", "Count"]


#Animated Line Chart
trace = go.Scatter(x=countrycount['Date'][0:2], y=countrycount['Count'][0:2],
                         mode = 'markers', line = dict(width = 2))

frames = [dict(data = [dict(type = 'scatter', x= countrycount['Date'][:k+1], y = countrycount['Count'][:k+1])],
               traces = [0,1], 
               ) for k in range(1, len(countrycount) - 1)
         ]
    
layout = go.Layout(width = 800, 
                   height = 450, 
                   showlegend = False, hovermode = 'closest', 
                    updatemenus=[dict(type='buttons', showactive=False,
                                y=1.05,
                                x=1.15,
                                xanchor='right',
                                yanchor='top',
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=30, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])])])
      

layout.update(xaxis =dict(range=[countrycount.Date[0], countrycount.Date[len(countrycount)-1]], 
                          autorange=False, showgrid = True, showline = True,
                          showticklabels=True,
                          linecolor = 'rgb(204, 204, 204)', 
                          linewidth = 2
                         ),
              yaxis =dict(range=[min(countrycount.Count)-10, max(countrycount.Count)+20], 
                          autorange=False, showline = True,
                          showticklabels=True,
                          linecolor = 'rgb(204, 204, 204)', 
                          linewidth = 2
                         ), 
              title = "Number Of Countries Affected Over Time"
             );
fig = go.Figure(data=[trace], frames=frames, layout=layout)
fig.show()



### Confirmed Cases - Race Chart
*Created using Flourish Studio. Data as of 04/07 *

URL - https://public.flourish.studio/visualisation/1873703/

<hr>

In [ ]:
#Race Chart
HTML('''<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1873703" data-url="https://flo.uri.sh/visualisation/1873703/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>''')

### Summary For Countries (sorted on confirmed count)
<hr>

In [ ]:
#Visualize Count Data

#Get latest data
dflatest = df[df['Date'] == max(df['Date'])]

#Aggregate Values
grouped = dflatest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum()
summary = grouped.sort_values(by = 'Confirmed', ascending = False).reset_index()

#Custom formatting
summary.style.background_gradient(cmap= 'Blues', subset = ['Confirmed'])\
             .background_gradient(cmap= 'Reds', subset = ['Deaths'])\
             .background_gradient(cmap= 'Greens', subset = ['Recovered'])\
             .background_gradient(cmap = 'Oranges', subset = ['Active'])

### World Trend Plot

In [ ]:
#Function for trend plot
def trendplot(data, xaxis, lables, title, line_size, mode_size, colors):
    fig = go.Figure()

    for i in range(len(labels)):
        fig.add_trace(go.Scatter(x=data[xaxis], y=data[labels[i]], mode='lines+markers',
            name=labels[i],
            line=dict(color=colors[i], width=line_size[i]),
            connectgaps=True
        ))

    fig.update_layout(
        xaxis = dict(
                showline = True, 
                showgrid = True, 
                showticklabels = True,
                linecolor = 'rgb(204, 204, 204)', 
                linewidth = 2, 
        ticks = 'inside'), 

        yaxis=dict(
            showgrid=True,
            zeroline=False,
            showline=False,
            showticklabels=True,
            linecolor = 'rgb(204, 204, 204)', 
            linewidth = 2
        ),

        autosize = True, 
        title = title

    )
    fig.show()



In [ ]:

#World Trend Plot
worldsummary = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index() 

xaxis = 'Date'
title = 'World Trend'
labels = ['Confirmed', 'Deaths', 'Recovered', 'Active']
colors = ['#51C1F9', '#FF4B4B', '#2ECC71', '#ECF0F1']
mode_size = [1,1,1,1]
line_size = [3,3,3,3]  

trendplot(data = worldsummary, xaxis = xaxis, 
          lables = labels, title = title, 
          line_size = line_size, mode_size = mode_size, 
          colors = colors)


In [ ]:
#Choropleth Map for Confirmed

fig = px.choropleth(dflatest, locations="Country/Region",
                    color=np.log10(dflatest["Confirmed"]), 
                    hover_name="Country/Region", 
                    hover_data=["Confirmed"],
                    color_continuous_scale=px.colors.sequential.Plasma,locationmode="country names")
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title="Confirmed Cases Heat Map (Log Scale)")
fig.update_coloraxes(colorbar_title="Confirmed Cases(Log Scale)",colorscale="Blues")
# fig.to_image("Global Heat Map confirmed.png")
fig.show()



In [ ]:
#Choropleth Map for Deaths

fig = px.choropleth(dflatest, locations="Country/Region",
                    color=np.log10(dflatest["Deaths"]), 
                    hover_name="Country/Region", # column to add to hover information
                    hover_data=["Deaths"],
                    color_continuous_scale=px.colors.sequential.Plasma,locationmode="country names")
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title ="Death Cases Heat Map (Log Scale)")
fig.update_coloraxes(colorbar_title="Death Cases(Log Scale)",colorscale="Reds")
# fig.to_image("Global Heat Map confirmed.png")
fig.show()



In [ ]:
#Choropleth Map for Recovered

fig = px.choropleth(dflatest, locations = 'Country/Region',
                    color = np.log10(dflatest['Recovered']), 
                   hover_name = "Country/Region", 
                   hover_data = ["Recovered"], 
                   color_continuous_scale = px.colors.sequential.Plasma, locationmode = "country names")

fig.update_layout(title ="Recovered Cases Heat Map (Log Scale)" )
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_coloraxes(colorbar_title="Recovered Cases(Log Scale)", colorscale = "Greens")
fig.show()

In [ ]:
#Scatter Geo for Confirmed

data = df.groupby(['Date','Country/Region'])['Confirmed', 'Deaths'].sum().reset_index()
data["Date"] = pd.to_datetime( data["Date"]).dt.strftime('%m/%d/%Y')

fig = px.scatter_geo(data, locations = "Country/Region", locationmode = "country names", 
                     color= np.power(data["Confirmed"],0.2)- 0.5 , size= np.power(data["Confirmed"]+1,0.3)-1, hover_name="Country/Region",
                     hover_data=["Confirmed"],
                     range_color= [0, max(np.power(data["Confirmed"],0.2))], 
                     projection="natural earth", animation_frame="Date", 
                     color_continuous_scale=px.colors.sequential.Plasma,
                     title = "Time Lapse of Confirmed Cases"
                    )

fig.update_coloraxes(colorscale = 'hot')
fig.update(layout_coloraxis_showscale = False)
fig.show()




In [ ]:
#Scatter Geo for Deaths

data = df.groupby(['Date','Country/Region'])['Confirmed', 'Deaths'].sum().reset_index()
data["Date"] = pd.to_datetime( data["Date"]).dt.strftime('%m/%d/%Y')

fig = px.scatter_geo(data, locations = "Country/Region", locationmode = "country names", 
                     color= 8.8 - np.power(data["Deaths"],0.2) , size= np.power(data["Deaths"]+1,0.3)-1, hover_name="Country/Region",
                     hover_data=["Deaths"],
                     range_color= [0, max(np.power(data["Deaths"],0.2))], 
                     projection="natural earth", animation_frame="Date", 
                     color_continuous_scale=px.colors.sequential.Plasma,
                     title = "Time Lapse of Death Numbers"
                    )

fig.update_coloraxes(colorscale = 'hot')
fig.update(layout_coloraxis_showscale = False)
fig.show()




In [ ]:
#Cal Map


f = plt.figure(figsize=(20,10))
f.add_subplot(2,1,1)
calmap.yearplot(df.groupby('Date')['Confirmed'].sum().diff(), fillcolor='White', cmap='GnBu', linewidth=1,linecolor="#fafafa",year=2020,)
plt.title("Daily Confirmed Cases",fontsize=20)
plt.tick_params(labelsize=15)


### Exploring US Counties
<hr>

In [ ]:
#Exloring US Data 
US.info()
US.head()


In [ ]:
#Top 20 States

#Get latest data
USlatest = US[US['Date'] == max(US['Date'])]

#Aggregate Values
grouped = USlatest.groupby('Province_State')['Confirmed', 'Deaths', 'Active'].sum()
summary = grouped.sort_values(by = 'Active', ascending = False)[:20].reset_index()

#Custom formatting
summary.style.background_gradient(cmap= 'Blues', subset = ['Confirmed'])\
             .background_gradient(cmap= 'Reds', subset = ['Deaths'])\
             .background_gradient(cmap = 'Oranges', subset = ['Active'])

In [ ]:
#Growth for the top 5 States

top5states = summary['Province_State'][:5].to_list()
top5states

#Plot Trend Graph for the top 5 states
ustrend =  US[US['Province_State'].isin(top5states)].groupby(['Date','Province_State'])['Confirmed', 'Deaths', 'Active'].sum().reset_index() 
#Params

fig = px.line(ustrend, x="Date", y="Active", color="Province_State",
              line_group="Province_State", hover_name="Province_State")
fig.update_layout(
        xaxis = dict(
                showline = True, 
                showgrid = True, 
                showticklabels = True,
                linecolor = 'rgb(204, 204, 204)', 
                linewidth = 2, 
        ticks = 'outside'), 

        yaxis=dict(
            showgrid=True,
            zeroline=False,
            showline=True,
            showticklabels=True,
            linecolor = 'rgb(204, 204, 204)', 
            linewidth = 2
        ))
fig.show()

### Top US Counties
<hr>

In [ ]:
nyt_us_counties.info()

#Change "date" field to datetime
   
nyt_us_counties['date'] = pd.to_datetime(nyt_us_counties['date'])

In [ ]:
nyt_us_counties_recent = nyt_us_counties[nyt_us_counties.date == max(nyt_us_counties.date)]

#Top 20 Counties
top20counties = nyt_us_counties_recent.groupby(['county', 'state'])['cases', 'deaths'].sum().sort_values('cases', ascending = False)[:20].reset_index()
top20counties.style.background_gradient(cmap = 'RdGy')

